In [1]:
print('all ok')

all ok


In [2]:
%pwd

'c:\\Users\\Bhavj\\Documents\\Healthwise-ChatBot\\research'

In [3]:
import os
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Bhavj\\Documents\\Healthwise-ChatBot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [6]:
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [7]:
extracted_data = load_pdf_files("data")

In [8]:
len(extracted_data)

637

In [9]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Documents objects, return a new list of document objects,
    containing only 'source' in metadeta and the original page content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get('source')
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={'source': src}
            )
        )
    return minimal_docs

In [10]:
minimal_docs=filter_to_minimal_docs(extracted_data)

In [11]:
minimal_docs[0].metadata

{'source': 'data\\Medical_book.pdf'}

In [12]:
# split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [13]:
texts_chunk=text_split(minimal_docs)
print(f'Number of Chunks: {len(texts_chunk)}')


Number of Chunks: 5859


## Embeddings


In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download the embeddings model from HuggingFace.
    """
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
    )
    return embeddings

embeddings = download_embeddings()

C:\Users\Bhavj\AppData\Local\Temp\ipykernel_18436\1019303769.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\Bhavj\Documents\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Bhavj\Documents\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook

In [15]:
vector = embeddings.embed_query('hello world')

In [16]:
vector

[-0.03447727486491203,
 0.03102312609553337,
 0.006734980270266533,
 0.026108933612704277,
 -0.03936205804347992,
 -0.16030246019363403,
 0.06692394614219666,
 -0.006441438104957342,
 -0.047450482845306396,
 0.014758863486349583,
 0.07087534666061401,
 0.05552757531404495,
 0.019193356856703758,
 -0.02625126577913761,
 -0.01010954286903143,
 -0.026940442621707916,
 0.022307462990283966,
 -0.02222665585577488,
 -0.14969263970851898,
 -0.017493024468421936,
 0.007676282897591591,
 0.054352231323719025,
 0.0032544038258492947,
 0.03172588348388672,
 -0.08462139964103699,
 -0.029405992478132248,
 0.051595550030469894,
 0.048124078661203384,
 -0.003314835485070944,
 -0.05827915295958519,
 0.04196925833821297,
 0.022210702300071716,
 0.1281888633966446,
 -0.022338951006531715,
 -0.011656239628791809,
 0.06292837113142014,
 -0.03287634998559952,
 -0.09122604131698608,
 -0.03117534890770912,
 0.052699536085128784,
 0.04703483358025551,
 -0.08420310169458389,
 -0.030056182295084,
 -0.0207448396

In [17]:
print('Vector length:',len(vector))

Vector length: 384


In [18]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [19]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [20]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [21]:
pc

In [22]:
from pinecone import Pinecone, ServerlessSpec

index_name = "healthwise-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    
index = pc.Index(index_name)

In [23]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embeddings,
    index_name=index_name
    
)

In [33]:
# load existing index
from langchain_pinecone import PineconeVectorStore
#embed each chunk and upsert the embeddings
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

## Add more data to the existing Pinecone index


In [34]:
bhavj=Document(
    page_content="This is a test document where i provide machine learning and rag tutorials.",
    metadata={"source": "test_source.pdf"}
)

In [35]:
docsearch.add_documents(documents=[bhavj])

['4f445a42-ace7-43eb-bae4-a75c2f7ca76f']

In [36]:
retriever =  docsearch.as_retriever(search_type='similarity', search_kwargs={"k": 3})

In [37]:
retrieved_docs=retriever.invoke('Example of chronic obstructive pulmonary disease?')
retrieved_docs

[Document(id='41cafdd4-ecce-4b81-982a-c6a13d50169f', metadata={'source': 'data\\Medical_book.pdf'}, page_content='slow onset over a long period of time.\nChronic obstructive pulmonary disease (COPD)—\nA term used to describe chronic lung diseases, like\nchronic bronchitis, emphysema, and asthma.\nEmphysema —One of the several diseases called\nchronic obstructive pulmonary diseases, emphyse-\nma involves the destruction of air sac walls to form\nabnormally large air sacs that have reduced gas\nexchange ability and that tend to retain air within\nthe lungs. Symptoms include labored breathing,'),
 Document(id='7f37f7db-658b-4f2d-9890-7e6b59d02321', metadata={'source': 'data\\Medical_book.pdf'}, page_content='slow onset over a long period of time.\nChronic obstructive pulmonary disease (COPD)—\nA term used to describe chronic lung diseases, like\nchronic bronchitis, emphysema, and asthma.\nEmphysema —One of the several diseases called\nchronic obstructive pulmonary diseases, emphyse-\nma i

In [68]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

# Load API key
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

chat = ChatGroq(
    model="llama3-8b-8192",   # or "llama3-70b-8192"
    api_key=groq_api_key
)



In [69]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [70]:
system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"

)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
        
    ]
)

In [71]:
question_answer_chain = create_stuff_documents_chain(chat, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [72]:
response = rag_chain.invoke({'input':"What is acromegaly and gigantism?"})
print(response['answer'])

Acromegaly is a disorder caused by the abnormal release of a chemical from the pituitary gland, leading to increased growth in bone and soft tissue, as well as various other bodily disturbances.
